# Natural Language Processing is Fun!
## An NLP workshop by Adam Geitgey

* References:

https://medium.com/@ageitgey/natural-language-processing-is-fun-9a0bff37854e

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/learning-stack/Colab-ML-Playbook/blob/master/NLP/Natural%20Language%20Processing%20is%20Fun/NLP_Is_Fun.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/learning-stack/Colab-ML-Playbook/blob/master/NLP/Natural%20Language%20Processing%20is%20Fun/NLP_Is_Fun.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [0]:
### Installing spaCy, general Python NLP lib
!pip3 install spacy

### Downloading the English dictionary model for spaCy
!python3 -m spacy download en_core_web_lg

### Installing textacy, basically a useful add-on to spaCy
!pip3 install textacy